In [ ]:
import os
import re
import sys
import json
import glob
import git
import xarray
import datetime
import rasterio
import tifffile
import subprocess
import numpy as np

from matplotlib import pyplot as plt

sys.path.insert(0, '../')
from managers import utils, managers

%load_ext autoreload
%autoreload 2

In [ ]:
# band 3 (veggie) from daytime 10/5 (DOY 278 at UTC 20:00)
'/home/keith/Downloads/OR_ABI-L1b-RadC-M6C03_G17_s20192782201196_e20192782203569_c20192782204022.nc'

In [ ]:
# 2019-08-31 at 7AM
# (note that band 2 looks cool using plt.imshow(utils.autoscale(im, minn=155, maxx=333, gamma=.7, dtype='uint8'), cmap='gray'))
filepaths = glob.glob('/home/keith/raster-data/GOES-R/2019-08-31_07-01-AM-PST/*.nc')

In [ ]:
# 2019-10-03 at 5PM
filepaths = glob.glob('/home/keith/raster-data/GOES-R/2019-10-03_05-01-PM-PST/*.NC')

In [ ]:
# 2019-10-05 at 6PM
filepaths = glob.glob('/home/keith/raster-data/GOES-R/2019-10-05_06-01-PM-PST/*.nc')

### Loading the data using xarray (fast but projection-less)

In [ ]:
d = xarray.open_dataset(filepaths[1])
im = d['Rad'].data
im.shape

In [ ]:
list(d.keys())

In [ ]:
d['goes_imager_projection']

### Creating a GeoTIFF using rio warp

For some reason, this is very slow (1-2min) for band 2.

In [ ]:
# create TIFFs from the netCDF files
# (note that this is pretty slow for band 2, even if we specify a small ROI)
for filepath in filepaths:
    utils.run_command([
        'rio', 'warp',
        'NETCDF:%s:Rad' % filepath,
        '%s.tif' % filepath])

In [ ]:
im = tifffile.imread('%s.tif' % filepaths[0])
im.dtype, im.shape, im.min(), im.max()

In [ ]:
plt.figure(figsize=(24,12))
plt.imshow(utils.autoscale(im[::-2, ::2], minn=0, maxx=100), cmap='gray')

In [ ]:
# bounds for central Sierra 
latlon_bounds = [-119.8, 37.2, -117.8, 38.2]

# bounds for west coast to SLC
latlon_bounds = [-125, 33, -105, 45]

south_sierra = [-121, 36, -116, 39]

In [ ]:
p = managers.GOESProject(
    project_root='/home/keith/raster-data/GOES-R/2019-08-31_07-01-AM-PST-PROJECT-01/',
    dataset_paths='/home/keith/raster-data/GOES-R/2019-08-31_07-01-AM-PST/',
    reset=True)

In [ ]:
p.raw_datasets[0].extant_bands

In [ ]:
p.warp(p.raw_datasets, res=500, bounds=, crs='EPSG:3857')

In [ ]:
imR = tifffile.imread(p.get_operation(-2).destination.filepath(2))
imG = tifffile.imread(p.get_operation(-2).destination.filepath(3))
imB = tifffile.imread(p.get_operation(-2).destination.filepath(1))

imR.dtype, imR.shape, imR.min(), imR.max()

In [ ]:
imG_adj = .45*imR + .1*imG + .45*imB
imG_blend = .7*imG_adj + .3*imG

In [ ]:
# 99th percentile for south sierra ROI for 2019-10-05_06-01-PM-PST
{'r': array([159., 289.]),
 'g': array([35., 99.]),
 'g_adj': array([ 98.65, 167.5 ]),
 'g_blend': array([ 80.525  , 143.58899]),
 'b': array([50., 69.])}

In [ ]:
{
'r': np.percentile(imR, (.1, 99.9)),
'g': np.percentile(imG, (.1, 99.9)),
'g_adj': np.percentile(imG_adj, (.1, 99.9)),
'g_blend': np.percentile(imG_blend, (.1, 99.9)),
'b': np.percentile(imB, (.1, 99.9)),
}

In [ ]:
imRGB = np.concatenate((
        utils.autoscale(imR, minn=160, maxx=290, gamma=.7, dtype='uint8')[:, :, None], 
        utils.autoscale(imG_adj, minn=98, maxx=167, gamma=.7, dtype='uint8')[:, :, None], 
        utils.autoscale(imB, minn=50, maxx=70, gamma=.9, dtype='uint8')[:, :, None],
    ), axis=2)

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(imRGB)

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(utils.autoscale(im, minn=155, maxx=333, gamma=.7, dtype='uint8'), cmap='gray')

In [ ]:
tifffile.imwrite('tmp_R-Gadj-B.tif', imRGB)